# Model Evaluation Script

In [233]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, ActivityRegularization
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping, TensorBoard, ProgbarLogger
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


from keras import backend as K
from sklearn.metrics import log_loss




# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../March-Madness"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Data
Data_Organization.ipynb
Final
Final_Data_Organization.ipynb
Final MLP.ipynb
GiantKillerInfo.txt
logs
MarchMadnessAdvStats.csv
MarchMadnessFeatureDifferences.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
ModelEvaluation.ipynb
OldNbs
README.md
SubmissionData.py
Submissions
tENSORFLOW.ipynb
tENSORFLOW.py
tENSORFLOW-TB.ipynb



Lock Random Seed

In [245]:
# seed = 99
np.random.seed()

## Load pre-organized data

In [345]:
data_dir = '../March-Madness/'
df_features = pd.read_csv(data_dir + 'MarchMadnessFeatures.csv')

Format and scale the data

In [346]:
X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X_train = X.values.reshape(-1,xDim)
y_train = df_features.Result.values

scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)


print('Feature vector dimension is: %.2f' % xDim)
# print(X[:5, :])

Feature vector dimension is: 34.00


In [347]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)

In [348]:
xDim = np.shape(X_train)[1]
xDim

2

# Load Training Data


In [349]:
df_test = pd.read_csv(data_dir+'MarchMadnessTest.csv')

X_test = df_test.iloc[:,1:]
xDimTest = np.shape(X_test)[1]
X_test = X_test.values.reshape(-1,xDimTest)
y_test = df_test.Result.values

In [350]:
# X_test = scaler.transform(X_test)
X_test = pca.transform(X_test)


In [351]:
np.shape(X_test)

(402, 2)

Create Kfold splits

In [352]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

## Logisitic Regression

In [353]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=18)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {} '.format(clf.best_score_, 
                                                            clf.best_params_['C']))

cvresults = cross_val_score(logreg, X_train, y_train, cv=kfold ,scoring='neg_log_loss', verbose=0)
print('Log-Loss Mean :{:.3} ({:.3})'.format(np.mean(cvresults), np.std(cvresults)))

print("Training set score: %f" % clf.score(X_train, y_train))
print("Validation set score: %f" % clf.score(X_test, y_test))

Best log_loss: -0.5952, with best C: 8.733261623828437e-05 
Log-Loss Mean :-0.601 (0.0523)
Training set score: -0.592216
Validation set score: -0.551134


## Random Forest Classifier

In [358]:
fore = RandomForestClassifier(criterion='gini', n_estimators = 50, max_depth = 4)

fore.fit(X_train, y_train)

cvresults = cross_val_score(fore, X_train, y_train, cv=kfold ,scoring='neg_log_loss', verbose=0)
print('Log-Loss Mean :{:.3} ({:.3})'.format(np.mean(cvresults), np.std(cvresults)))

y_pred =  fore.predict_proba(X_train)[:,1].reshape(-1,1)
LL = log_loss( y_train, y_pred)
print("Training set score: {:4}" .format(LL))
y_pred =  fore.predict_proba(X_test)[:,1].reshape(-1,1)
LL = log_loss( y_test, y_pred)
print("Validation set score: {:4}".format(LL))

Log-Loss Mean :-0.619 (0.0164)
Training set score: 0.5432213305942657
Validation set score: 0.5492697107341292


## MLP

Various properties

In [359]:
dropRate = 0.3
numBatch = 5
numEpoch = 120
learningRate = 1e-4

Base Single Layer Model

In [360]:
# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(200, input_dim=xDim, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200,kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
MLP.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [361]:
# TB = TensorBoard(log_dir="logs/{}_{}{}".format(MLP.name, localtime().tm_hour, localtime().tm_min))
# MLP.fit(X, y, validation_split = 0.2, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)


Epoch 1/120
530/530 [==============================] - 2s 4ms/step - loss: 0.6720 - acc: 0.6415
Epoch 2/120
530/530 [==============================] - 0s 424us/step - loss: 0.6456 - acc: 0.6472
Epoch 3/120
530/530 [==============================] - 0s 418us/step - loss: 0.6448 - acc: 0.6660
Epoch 4/120
530/530 [==============================] - 0s 478us/step - loss: 0.6329 - acc: 0.6774
Epoch 5/120
530/530 [==============================] - 0s 460us/step - loss: 0.6262 - acc: 0.6660
Epoch 6/120
530/530 [==============================] - 0s 466us/step - loss: 0.6212 - acc: 0.6811
Epoch 7/120
530/530 [==============================] - 0s 397us/step - loss: 0.6380 - acc: 0.6849
Epoch 8/120
530/530 [==============================] - 0s 428us/step - loss: 0.6325 - acc: 0.6868
Epoch 9/120
530/530 [==============================] - 0s 431us/step - loss: 0.6385 - acc: 0.6660
Epoch 10/120
530/530 [==============================] - 0s 408us/step - loss: 0.6255 - acc: 0.6849
Epoch 11/120
530/530 

530/530 [==============================] - 0s 388us/step - loss: 0.5981 - acc: 0.6830
Epoch 84/120
530/530 [==============================] - 0s 410us/step - loss: 0.5878 - acc: 0.6679
Epoch 85/120
530/530 [==============================] - 0s 377us/step - loss: 0.5970 - acc: 0.6830
Epoch 86/120
530/530 [==============================] - 0s 371us/step - loss: 0.6055 - acc: 0.6811
Epoch 87/120
530/530 [==============================] - 0s 396us/step - loss: 0.5978 - acc: 0.6623
Epoch 88/120
530/530 [==============================] - 0s 391us/step - loss: 0.6106 - acc: 0.6717
Epoch 89/120
530/530 [==============================] - 0s 376us/step - loss: 0.6108 - acc: 0.6528
Epoch 90/120
530/530 [==============================] - 0s 380us/step - loss: 0.6120 - acc: 0.6736
Epoch 91/120
530/530 [==============================] - 0s 382us/step - loss: 0.6125 - acc: 0.6660
Epoch 92/120
530/530 [==============================] - 0s 395us/step - loss: 0.5988 - acc: 0.6943
Epoch 93/120
530/530 [=

In [362]:
MLP.evaluate(X_test, y_test)

402/402 [==============================] - 1s 2ms/step


[0.5368121343465587, 0.7462686576060394]

## SVM

In [101]:
# svc = SVC(C = .01, kernel = 'linear', probability = True)
svm = SVC(probability=True)
svc = GridSearchCV(svm, param_grid={
     'C': [0.001, 0.01, 0.1, 1, 10],
      'gamma': [0.001, 0.01, 0.1, 1, 10]},
      scoring ='neg_log_loss', refit=True)

svc.fit(X_train,y_train)
print('Best MSE: {:.4}, with best C: {} and best gamma: {}'.format(svc.best_score_, 
                                                                        svc.best_params_['C'], svc.best_params_['gamma']))
cvresults = cross_val_score(svc, X_train, y_train, cv=kfold ,scoring='neg_log_loss', verbose=0)
print('Log-Loss Mean :{:.3} ({:.3})'.format(np.mean(cvresults), np.std(cvresults)))

print("Training set score: %f" % svc.score(X_train, y_train))
print("Validation set score: %f" % svc.score(X_test, y_test))

Best MSE: -0.6231, with best C: 1 and best gamma: 0.001
Log-Loss Mean :-0.634 (0.0199)
Training set score: -0.596728
Validation set score: -0.587577


## Making predictions with desired model

### Extract data desired

In [30]:
data_dir = '../NCAA/Data/'
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
data_file = data_dir + 'MarchMadnessAdvStats.csv'
df_adv = pd.read_csv(data_file)
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')


n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def seed_to_int(seed):
    '''Get just the digits from the seeding. Return as int'''
    s_int = int(seed[1:3])
    return s_int

In [ ]:
print('Loading data for submission test')

# Make the seeding an integer
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label
df_seeds.head()


T1_seed = []
T1_adv = []
T2_adv = []
T2_seed = []
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    t1_adv = df_adv[(df_adv.TeamID == t1) & (df_adv.Season == year)].values[0]
    t2_adv = df_adv[(df_adv.TeamID == t2) & (df_adv.Season == year)].values[0]
    T1_seed.append(t1_seed)
    T1_adv.append(t1_adv)
    T2_seed.append(t2_seed)
    T2_adv.append(t2_adv)

T1_adv = [row[2:] for row in T1_adv]
T2_adv = [row[2:] for row in T2_adv]
T1_seed = np.reshape(T1_seed, [n_test_games,-1]).tolist()
T2_seed = np.reshape(T2_seed, [n_test_games, -1]).tolist()
X_pred = np.concatenate((T1_seed, T1_adv, T2_seed, T2_adv), axis=1)

df_subData = pd.DataFrame(np.array(X_pred).reshape(np.shape(X_pred)[0], np.shape(X_pred)[1]))

xDim = np.shape(df_subData)[1]
X_pred = df_subData.values.reshape(-1,xDim)

In [36]:
np.shape(T1_seed)

(9112, 1)

In [33]:
# X_test = scaler.fit_transform(X_test)

#s*************Check if this works!!!
preds = MLP.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(9112, 2)

In [26]:
filename = 'MLP'
save_dir = '../NCAA/Submissions/'
c=0
ext = '.csv'
if os.path.exists(save_dir+filename+ext):
    while os.path.exists(filename+ext):
        c+=1
    filename = filename+'_'+str(c)
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)
else:
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)